## Goal of notebook

    Look at boundary conditions for west, north
    parametrize according to salinity relationships for JdF and northern strait from nb
    make a new init file
    
    where does open_boundaries lead to? it must be:
        /home/tjarnik/MEOPAR/tracers/north
            bioOBC_full2_north.nc
        /home/tjarnik/MEOPAR/tracers/west
    but where is the link?
            bioOBC_full2_west.nc
 
    
    
    relationships DIC (JdF, N, S, Haro:
    slopes:
    [68.984506591824541, 75.046682176725568, 45.248982910520937, 62.275868280536201]
    intercepts:
    [-100.13978357455562, -210.80778641253437, 669.0670004789929, 128.20407948614184]

    relationships alkalinity (JdF, N, S, Haro):
    slopes:
    [46.987083509452937, 41.703959570158162, 45.255106772714242, 48.099718320884534]
    intercepts:
    [666.6180299218554, 805.39142381007036, 712.25447700734412, 630.84821673141755]

currently initializing to NO3

In [12]:
#import all the things

%reset
import netCDF4 as nc
import matplotlib.pyplot as plt
import datetime
import os
import numpy as np
import cmocean as cm
from salishsea_tools import visualisations as vis
from salishsea_tools import (teos_tools, tidetools, viz_tools)
import xarray as xr 
import subprocess 
import cmocean

%matplotlib inline

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [13]:
d_nor = '/data/tjarniko/MEOPAR/tracers/north/'
f_nor = 'bioOBC_full2_north.nc'
phys_nor = 'SalishSea_north_TEOS10.nc'
d_wes = '/data/tjarniko/MEOPAR/tracers/west/'
f_wes = 'bioOBC_full2_west.nc'
phys_wes = 'SalishSea_west_TEOS10.nc'


tr_north = nc.Dataset(d_nor+f_nor)
tr_west = nc.Dataset(d_wes+f_wes)
phys_north = nc.Dataset(d_nor+phys_nor)
phys_west = nc.Dataset(d_wes+phys_wes)

In [14]:
print('tracers north')
print(tr_north)
print('tracers west')
print(tr_west)
print('physics north')
print(phys_north)
print('physics west')
print(phys_west)

tracers north
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): xbT(300), yb(1), deptht(40), time_counter(2)
    variables(dimensions): float32 deptht(deptht), float32 nav_lat(yb,xbT), float32 nav_lon(yb,xbT), int32 nbidta(yb,xbT), int32 nbjdta(yb,xbT), int32 nbrdta(yb,xbT), float32 time_counter(time_counter), float32 NO3(time_counter,deptht,yb,xbT), float32 Si(time_counter,deptht,yb,xbT), float32 NH4(time_counter,deptht,yb,xbT), float32 DIA(time_counter,deptht,yb,xbT), float32 CRY(time_counter,deptht,yb,xbT), float32 MYRI(time_counter,deptht,yb,xbT), float32 MICZ(time_counter,deptht,yb,xbT), float32 PON(time_counter,deptht,yb,xbT), float32 DON(time_counter,deptht,yb,xbT), float32 bSi(time_counter,deptht,yb,xbT), float32 O2(time_counter,deptht,yb,xbT), float32 tur(time_counter,deptht,yb,xbT)
    groups: 

tracers west
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): de

In [15]:
sn_nor = phys_north.variables['vosaline']
print('northern salinity')
#print(sn_nor)
#print(sn_nor[0,2,:,:])
sn_wes = phys_west.variables['vosaline']
print('western salinity')
print(sn_wes)
print(sn_wes[0,2,:,0])

print(tr_west)

NO3 = tr_west.variables['NO3']

#print(NO3[0,2,:,0:870:87])

northern salinity
western salinity
<class 'netCDF4._netCDF4.Variable'>
float32 vosaline(time_counter, deptht, yb, xbT)
    coordinates: nav_lon nav_lat deptht time_counter
    grid: SalishSea
    units: g/kg
    long_name: Reference Salinity
unlimited dimensions: time_counter
current shape = (52, 40, 10, 87)
filling on, default _FillValue of 9.969209968386869e+36 used

[ 29.95800018  29.95899963  29.95999908  29.95100021  29.95400047
  29.95599937  29.95700073  29.95800018  29.95800018  29.95899963]
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): deptht(40), yb(1), xbT(870), time_counter(52)
    variables(dimensions): float32 deptht(deptht), float32 nav_lat(yb,xbT), float32 nav_lon(yb,xbT), int32 nbidta(yb,xbT), int32 nbjdta(yb,xbT), int32 nbrdta(yb,xbT), float32 time_counter(time_counter), float32 NO3(time_counter,deptht,yb,xbT), float32 Si(time_counter,deptht,yb,xbT), float32 NH4(time_counter,deptht,yb,xbT), float32 DIA(tim

In [16]:
dsin = tr_west

#output file
ncname = 'tr_westDICTAOXY.nc'
dsout = nc.Dataset(ncname, "w", format="NETCDF4")
#Copy dimensions
for dname, the_dim in dsin.dimensions.items():
    print(dname)
    print(len(the_dim))
    dsout.createDimension(dname, len(the_dim) if not the_dim.isunlimited() else None)
# Copy variables
for v_name, varin in dsin.variables.items():
    outVar = dsout.createVariable(v_name, varin.datatype, varin.dimensions)
    print(varin.datatype)
    
    # Copy variable attributes
    outVar.setncatts({k: varin.getncattr(k) for k in varin.ncattrs()})
    
    outVar[:] = varin[:]

deptht
40
yb
1
xbT
870
time_counter
52
float32
float32
float32
int32
int32
int32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32


In [17]:
DIC = dsout.createVariable('DIC',np.float64,('time_counter','deptht','yb','xbT'))
TA = dsout.createVariable('TA',np.float64,('time_counter','deptht','yb','xbT'))
OXY = dsout.createVariable('OXY',np.float64,('time_counter','deptht','yb','xbT'))

In [18]:
# relationships DIC (JdF, N, S, Haro:
#     slopes:
#     [68.984506591824541, 75.046682176725568, 45.248982910520937, 62.275868280536201]
#     intercepts:
#     [-100.13978357455562, -210.80778641253437, 669.0670004789929, 128.20407948614184]

#     relationships alkalinity (JdF, N, S, Haro):
#     slopes:
#     [46.987083509452937, 41.703959570158162, 45.255106772714242, 48.099718320884534]
#     intercepts:
#     [666.6180299218554, 805.39142381007036, 712.25447700734412, 630.84821673141755]

# relationships OXY (Jdf, N, S, Haro:)
# slopes:
# [-1.1093358620615621, -1.0324527832500823, -0.72610938589358809, -0.74540955174472656]
# intercepts:
# [39.482801413558569, 35.238673873674337, 25.996793064314204, 27.695179997365347]

sn_wes_val = sn_wes[:]
DIC_wes_slope = 68.984506591824541
DIC_wes_int = -100.13978357455562

TA_wes_slope = 46.987083509452937
TA_wes_int = 666.6180299218554

OXY_wes_slope = -1.1093358620615621
OXY_wes_int = 39.482801413558569

DIC_wes = sn_wes_val*DIC_wes_slope + DIC_wes_int
TA_wes = sn_wes_val*TA_wes_slope + TA_wes_int
OXY_wes = sn_wes_val*OXY_wes_slope + OXY_wes_int

print('western DIC')
print(DIC_wes[:,:,0,0])
print('western TA')
print(TA_wes[:,:,0,0])
print('western OXY')
print(OXY_wes[:,:,0,0])

western DIC
[[ 1956.4263916   1961.59997559  1966.4979248  ...,  2185.31689453
   2185.31689453  2185.31689453]
 [ 1946.83740234  1952.28723145  1957.39196777 ...,  2182.35058594
   2182.35058594  2182.35058594]
 [ 1941.80163574  1948.01013184  1953.94274902 ...,  2193.87084961
   2193.87084961  2193.87084961]
 ..., 
 [ 1976.50085449  1981.95056152  1987.67614746 ...,  2198.49267578
   2198.49267578  2198.49267578]
 [ 1979.12219238  1984.0201416   1988.91809082 ...,  2193.25        2193.25
   2193.25      ]
 [ 1969.25744629  1974.22424316  1979.05310059 ...,  2187.93798828
   2187.93798828  2187.93798828]]
western TA
[[ 2067.39697266  2070.92089844  2074.25708008 ...,  2223.30029297
   2223.30029297  2223.30029297]
 [ 2060.86572266  2064.57763672  2068.0546875  ...,  2221.27978516
   2221.27978516  2221.27978516]
 [ 2057.43554688  2061.66455078  2065.70556641 ...,  2229.12646484
   2229.12646484  2229.12646484]
 ..., 
 [ 2081.0703125   2084.78222656  2088.68212891 ...,  2232.2746582
  

In [19]:
no3 = tr_west.variables['NO3']
print(no3.shape)
print(DIC_wes.shape)

DIC_resized = no3[:]
TA_resized = no3[:]
OXY_resized = no3[:]


for i in range(0,10):
    start = i * 87 
    end = (i+1) * 87
    DIC_resized[:,:,:,start:end] = DIC_wes[:,:,i:i+1,:]
    TA_resized[:,:,:,start:end] = TA_wes[:,:,i:i+1,:]
    OXY_resized[:,:,:,start:end] = OXY_wes[:,:,i:i+1,:]
    
    
# #print(DIC_resized[0,0,:,:])
# #print(sn_nor.shape)
# # print(DIC_nor[0,0,9,:])
# # print(sn_nor[0,0,9,:])

# print(no3.shape)

(52, 40, 1, 870)
(52, 40, 10, 87)


In [20]:
dsout['DIC'][:] = DIC_resized
dsout['TA'][:] = TA_resized
dsout['OXY'][:] = OXY_resized


In [21]:
dsout.close()

In [22]:
#w = nc.Dataset('./tr_northDICTA.nc')
w2 = nc.Dataset('./tr_westDICTAOXY.nc')
#print(w)
print(w2)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): deptht(40), yb(1), xbT(870), time_counter(52)
    variables(dimensions): float32 deptht(deptht), float32 nav_lat(yb,xbT), float32 nav_lon(yb,xbT), int32 nbidta(yb,xbT), int32 nbjdta(yb,xbT), int32 nbrdta(yb,xbT), float32 time_counter(time_counter), float32 NO3(time_counter,deptht,yb,xbT), float32 Si(time_counter,deptht,yb,xbT), float32 NH4(time_counter,deptht,yb,xbT), float32 DIA(time_counter,deptht,yb,xbT), float32 CRY(time_counter,deptht,yb,xbT), float32 MYRI(time_counter,deptht,yb,xbT), float32 MICZ(time_counter,deptht,yb,xbT), float32 PON(time_counter,deptht,yb,xbT), float32 DON(time_counter,deptht,yb,xbT), float32 bSi(time_counter,deptht,yb,xbT), float32 O2(time_counter,deptht,yb,xbT), float32 tur(time_counter,deptht,yb,xbT), float64 DIC(time_counter,deptht,yb,xbT), float64 TA(time_counter,deptht,yb,xbT), float64 OXY(time_counter,deptht,yb,xbT)
    groups: 



In [23]:
#print(w)